# 刻度数据

为了获得最佳结果，建议在外汇交易时间段运行这本笔记本。

In [1]:
from ib_async import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

### 实时 tick 数据

创建一些外汇合同：

In [2]:
contracts = [Forex(pair) for pair in ('EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD')]
ib.qualifyContracts(*contracts)

eurusd = contracts[0]

请求为他们提供流动的报价。

In [3]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

等几秒钟，让交易单填充完整。

In [4]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2019, 12, 31, 17, 5, 2, 127038, tzinfo=datetime.timezone.utc), bid=1.12245, bidSize=10700000, ask=1.1225, askSize=2000000, high=1.1239, low=1.11985, close=1.12, halted=0.0)

外汇ticks的价格始终是nan。要获得中间价格，请使用``midpoint()``或``marketPrice()``。

标记是保持实时更新的，请尝试几次看看价格是否会更改：

In [5]:
ticker.marketPrice()

1.1224750000000001

下一个单元格将启动一个30秒的循环，打印一个实时更新的股票表。在每次股票变动时更新。

In [6]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])

def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

,bidSize,bid,ask,askSize,high,low,close
EURUSD,11500000,1.1225,1.12255,1000000,1.1239,1.11985,1.12
USDJPY,8000000,108.665,108.675,16000000,108.885,108.475,108.87
GBPUSD,6000000,1.327,1.3271,500000,1.32845,1.3106,1.3111
USDCHF,10000000,0.9677,0.9678,7000000,0.9698,0.96465,0.96935
USDCAD,1000000,1.2966,1.29665,2000000,1.30695,1.29515,1.3068
AUDUSD,9500000,0.70305,0.7031,1000000,0.70325,0.69945,0.6995


新的tick数据可以在待处理股票的“ticks”属性中找到。
下一个更新之前，tick数据将被清除。

停止实时的勾选订阅：

In [7]:
for contract in contracts:
    ib.cancelMktData(contract)

### 逐笔数据 ###

前一节中的逐笔数据是由IB进行时间采样，以减少带宽使用。因此，使用“reqMktData”并不是从交易所发送每个逐笔数据。而“reqTickByTickData”的承诺是发送每个逐笔数据，就像在TWS时间和销售窗口中看到的那样。这个功能只能同时处理三个订阅，其中买卖盘和成交量也各使用一个订阅。

逐笔更新可以从“ticker.tickByTicks”中获取，并通过“ib.pendingTickersEvent”或“ticker.updateEvent”来接收信号。

In [8]:
ticker = ib.reqTickByTickData(eurusd, 'BidAsk')
ib.sleep(2)
print(ticker)

ib.cancelTickByTickData(ticker.contract, 'BidAsk')

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2019, 12, 31, 17, 5, 35, 432737, tzinfo=datetime.timezone.utc), bid=1.1225, bidSize=11000000, ask=1.1226, askSize=11500000, prevBid=1.12255, prevBidSize=11500000, prevAsk=1.12255, prevAskSize=1000000, high=1.1239, low=1.11985, close=1.12, halted=0.0, tickByTicks=[TickByTickBidAsk(time=datetime.datetime(2019, 12, 31, 17, 5, 35, 432737, tzinfo=datetime.timezone.utc), bidPrice=1.1225, askPrice=1.12255, bidSize=11500000, askSize=1000000, tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False)), TickByTickBidAsk(time=datetime.datetime(2019, 12, 31, 17, 5, 35, 432737, tzinfo=datetime.timezone.utc), bidPrice=1.1225, askPrice=1.12255, bidSize=11000000, askSize=1000000, tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False)), TickByTickBidAsk(time=datetime.datetime(2019, 12, 31, 17, 5, 35, 432737, tzinfo=datetime.timezone.utc)

### 历史tick数据

历史tick数据最多可以一次获取1000个ticks。必须提供开始时间或结束时间之一，其中一个必须保持为空：

In [9]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]

HistoricalTickBidAsk(time=datetime.datetime(2019, 12, 31, 17, 5, 34, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False), priceBid=1.1225, priceAsk=1.1226, sizeBid=11000000, sizeAsk=11500000)

In [10]:
ib.disconnect()